In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
242,AFG,Asia,Afghanistan,2020-08-29,38140.0,11.0,35.143,1402.0,1.0,2.429,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
416,ALB,Europe,Albania,2020-08-29,9195.0,112.0,153.714,271.0,5.0,4.429,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
659,DZA,Africa,Algeria,2020-08-29,43403.0,387.0,390.857,1483.0,8.0,9.286,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-28,France,263880.0,0.0,30597.0,0.0,0.0,0.0
0,2020-08-28,China,89848.0,0.0,4721.0,0.0,0.0,0.0
0,2020-08-28,Italy,265171.0,0.0,35495.0,0.0,0.0,0.0
0,2020-08-28,Spain,436270.0,0.0,29041.0,0.0,0.0,0.0
0,2020-08-28,United States,5950633.0,0.0,182080.0,0.0,0.0,0.0
0,2020-08-28,World,24935994.0,0.0,839923.0,0.0,0.0,0.0
0,2020-08-28,United Kingdom,331513.0,0.0,41526.0,0.0,0.0,0.0
0,2020-08-28,Germany,240853.0,0.0,9298.0,0.0,0.0,0.0
0,2020-08-28,Iran,370340.0,0.0,21332.0,0.0,0.0,0.0
0,2020-08-28,Turkey,265106.0,0.0,6226.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-29,France,263880.0,267077.0,30597.0,30596.0,-3197.0,1.0
0,2020-08-29,China,89848.0,89836.0,4721.0,4718.0,12.0,3.0
0,2020-08-29,Italy,265171.0,265409.0,35495.0,35472.0,-238.0,23.0
0,2020-08-28,Spain,436270.0,439286.0,29041.0,29011.0,-3016.0,30.0
0,2020-08-29,United States,5950633.0,5917439.0,182080.0,181773.0,33194.0,307.0
0,2020-08-29,World,24935994.0,24760129.0,839923.0,837579.0,175865.0,2344.0
0,2020-08-29,United Kingdom,331513.0,331644.0,41526.0,41486.0,-131.0,40.0
0,2020-08-29,Germany,240853.0,240986.0,9298.0,9289.0,-133.0,9.0
0,2020-08-29,Iran,370340.0,369911.0,21332.0,21249.0,429.0,83.0
0,2020-08-29,Turkey,265106.0,265515.0,6226.0,6245.0,-409.0,-19.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")